In [16]:
import numpy as np
import pandas as pd
import math
import sys

In [3]:
def extract_from_jscookies(cookies):
    set_cookies = cookies[["visit_id", "host", "name", "value", "time_stamp"]]
    
    # set cookies
    set_cookies["type"] = ["set"] * set_cookies.shape[0]
    return set_cookies

In [4]:
def sep_cookies(ls):
    res = []
    for l in ls:
        idx = l.find('=')
        if idx > 0:
            res.append((l[:idx], l[idx+1:]))
    return res


In [5]:
def extract_from_js(js):
    js_cookies = js[js.symbol == 'window.document.cookie']
    js_cookies = js_cookies[js_cookies.value.notnull()]
    js_cookies_list = list(map(lambda x:x.replace(" ","").split(';'), js_cookies.value.tolist()))
    js_cookies_ls_dict = list(map(lambda x:sep_cookies(x), js_cookies_list))

    js_cookies['cookies_dict'] = js_cookies_ls_dict

    js_cookies_temp = js_cookies[['visit_id','script_url', 'cookies_dict', "time_stamp"]]

    ids = []
    hosts = []
    c_names = []
    c_values = []
    t_stamps = []
    for _, row in js_cookies_temp.iterrows():
        Id = row.visit_id
        host = row.script_url
        for name, value in row.cookies_dict:
            ids.append(Id)
            hosts.append(host)
            c_names.append(name)
            c_values.append(value)
            t_stamps.append(row.time_stamp)

    js_cookies_new = pd.DataFrame({"visit_id":ids, "host": hosts, "name":c_names, "value":c_values, "time_stamp":t_stamps })

    # cookie use tables
    js_cookies_new["type"] = ["use"] * js_cookies_new.shape[0]
    return js_cookies_new

In [6]:
def extract_http(httprequests):
    httprequests = httprequests[httprequests.cookies.notnull()]

    ids = []
    hosts = []
    c_names = []
    c_values = []
    t_stamps = []
    for i, row in httprequests.iterrows():
        Id = row.visit_id
        cookies_ls = row.cookies.replace(' ','').split(';')
        for l in cookies_ls:
            idx = l.find('=')
            if idx > 0:
                ids.append(row.visit_id)
                hosts.append(row.host)
                c_names.append(l[:idx])
                c_values.append(l[idx+1:])
                t_stamps.append(row.time_stamp)

    httpcookies = pd.DataFrame({"visit_id":ids, "host": hosts, "name":c_names, "value":c_values, "time_stamp":t_stamps })

    httpcookies['type'] = ['send'] * httpcookies.shape[0]
    return httpcookies

In [7]:
def merge_cookies(jsc_cookies, js_cookies, http_cookies):
    bigcookies = pd.concat([jsc_cookies, js_cookies, http_cookies], ignore_index=True)
    return bigcookies

In [83]:
bigcookies.to_csv("bigcookies-1-3.csv", index=False)

In [183]:
##### concatenate

In [12]:
# cookies (set/change httpresponce, js)
bigcookies = []
experiment_num = sys.argv[0] #1
web_nums = sys.argv[1] #3
for i in range(1, web_nums+1):
    cookies = pd.read_csv("./tables/{}-{}-FP-cookies.csv".format(experiment_num, i))
    jscookiesc = extract_from_jscookies(cookies)
    httprequests = pd.read_csv("./tables/{}-{}-httprequest.csv".format(experiment_num, i))
    httpc = extract_http(httprequests)
    js = pd.read_csv("./tables/{}-{}-js.csv".format(experiment_num, i))
    jsc = extract_from_js(js)
    bigcookies.append(merge_cookies(jscookiesc, httpc, jsc))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [13]:
hugecookies = pd.concat(bigcookies)

In [86]:
hugecookies.to_csv("./hugecookies/hugecookies-{}.csv".format(experiment_num), index=False)